## Image feature extraction

In [2]:

from dgl.nn import SAGEConv
import dgl.function as fn
from umap.umap_ import UMAP
from sklearn.metrics import roc_auc_score
import dgl
import shutil
import torch
import torch.nn as nn
import torch.nn.functional as F
import itertools
import numpy as np
import scipy.sparse as sp
import json
import gzip
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from tqdm import tqdm
import dgl.data
import os
import random
import datetime
import tensorflow as tf
import os
import pandas as pd
import numpy as np
from PIL import Image
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image as keras_image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
import pickle

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
from mlflow import log_metric, log_param, log_artifact
import mlflow.sklearn
import mlflow.xgboost


from mlflow.tracking import MlflowClient


In [4]:
remote_server_uri = "http://127.0.0.1:5000/"  # set to your server URI
mlflow.set_tracking_uri(remote_server_uri)
experiment_name = 'Men_Clothing_Image_recommendation_v3'

mlflow.set_experiment(experiment_name)
# Forcing an end_run() to prevent 
#    https://github.com/mlflow/mlflow/issues/1335 
#    https://github.com/mlflow/mlflow/issues/608

mlflow.autolog()

artifact_path = mlflow.get_artifact_uri()
uri = mlflow.tracking.get_tracking_uri()
print(artifact_path)
print(uri)
mlflow.end_run()

2025/02/22 12:25:14 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2025/02/22 12:25:15 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/02/22 12:25:15 INFO mlflow.tracking.fluent: Autologging successfully enabled for keras.
2025/02/22 12:25:15 INFO mlflow.tracking.fluent: Autologging successfully enabled for tensorflow.
2025/02/22 12:25:15 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This ini

mlflow-artifacts:/473744221893913913/49d9fbce333e4b54abbe1066ee45aa69/artifacts
http://127.0.0.1:5000/


In [5]:
from mlflow.tracking import MlflowClient
client = MlflowClient(mlflow.get_tracking_uri())


In [6]:
IMAGE_MODEL = 'inception'

In [7]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

cuda
Num GPUs Available:  1


In [8]:
nodes = pd.read_parquet(f'../Dataset/nodes_without_image.parquet',engine='pyarrow')
edges = pd.read_parquet(f'../Dataset/edges_without_image.parquet',engine='pyarrow')

In [9]:
print("Node data shape :",nodes.shape)
print("Edges data shape :",edges.shape)
# print("Size of Input features :",len(cols_needed))
# print("node features shape :",node_features.shape)
# node_labels = torch.from_numpy(text_df['niche'].astype('category').cat.codes.to_numpy())
edges_src = torch.from_numpy(edges['asin'].to_numpy())
edges_dst = torch.from_numpy(edges['also_buy'].to_numpy())

Node data shape : (10867, 40)
Edges data shape : (20554, 2)


In [10]:
cols_needed = list(nodes.drop(columns=['asin','also_buy']).columns)

In [11]:
from util import *

In [12]:
IMAGE_MODEL = 'inception'
model,g,nodes,edges,train_pos_u, train_pos_v,test_pos_u, test_pos_v,train_neg_u, train_neg_v,test_neg_u, test_neg_v,train_g,test_g = train_model(mlflow=mlflow,image_model=IMAGE_MODEL,model_params = {'aggregator_type': 'lstm', 'dropout': 0.42051995958759714, 'h_feats': 16, 'lr': 0.022254491166903718},
                                                                                                                                          run_name=f'{IMAGE_MODEL}_no_features',
                                                                                                                                          n_epochs=100,
                                                                                                                                          image_included=False,
                                                                                                                                          split_ratio=0.3,umap_dt={'n_components':100,'n_neighbors':50,'min_dist':0.3},patience=100,only_image=False,skip_ndata=True,use_bidirectional=True)

skipping adding node features
Node data shape : (10867, 39)
Edges data shape : (20554, 2)
Size of Input features : 0
node features shape : torch.Size([10867, 39])
##########################
Converting to biderctional Graph
##########################
27998 11999
In epoch 0, loss: 27.480663299560547
Epoch 0: Best model saved min loss 27.480663299560547
Epoch 1: Best model saved min loss 8.636537551879883
Epoch 2: Best model saved min loss 4.44978666305542
Epoch 3: Best model saved min loss 2.210573673248291
Epoch 4: Best model saved min loss 0.9635067582130432
In epoch 5, loss: 0.6672428250312805
Epoch 5: Best model saved min loss 0.6672428250312805
Epoch 6: Best model saved min loss 0.571540892124176
Epoch 7: Best model saved min loss 0.5396424531936646
Epoch 8: Best model saved min loss 0.5073716044425964
Epoch 9: Best model saved min loss 0.46218207478523254
In epoch 10, loss: 0.429837167263031
Epoch 10: Best model saved min loss 0.429837167263031
Epoch 11: Best model saved min loss 0

100%|██████████| 10867/10867 [00:01<00:00, 6332.62it/s]


Train recall@100 0.6104862324571897
Test recall@100 0.5978765462443362


In [16]:
IMAGE_MODEL = 'inception'
model,g,nodes,edges,train_pos_u, train_pos_v,test_pos_u, test_pos_v,train_neg_u, train_neg_v,test_neg_u, test_neg_v,train_g,test_g = train_model(mlflow=mlflow,image_model=IMAGE_MODEL,model_params = {'aggregator_type': 'lstm', 'dropout': 0.42051995958759714, 'h_feats': 16, 'lr': 0.022254491166903718},
                                                                                                                                          run_name=f'{IMAGE_MODEL}_only_image',
                                                                                                                                          n_epochs=100,
                                                                                                                                          image_included=True,
                                                                                                                                          split_ratio=0.3,umap_dt={'n_components':100,'n_neighbors':50,'min_dist':0.3},patience=100,only_image=True,skip_ndata=True,use_bidirectional=True)

c:\Users\Kartik\anaconda3\envs\tf\lib\site-packages\umap\umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


Node data shape : (10867, 139)
Edges data shape : (20554, 2)
Size of Input features : 100
node features shape : torch.Size([10867, 100])
##########################
Converting to biderctional Graph
##########################
27998 11999
In epoch 0, loss: 374.6854553222656
Epoch 0: Best model saved min loss 374.6854553222656
Epoch 1: Best model saved min loss 45.380401611328125
Epoch 2: Best model saved min loss 1.8589144945144653
Epoch 3: Best model saved min loss 0.5453778505325317
Epoch 4: Best model saved min loss 0.4686381220817566
In epoch 5, loss: 0.42239633202552795
Epoch 5: Best model saved min loss 0.42239633202552795
Epoch 6: Best model saved min loss 0.38825875520706177
Epoch 7: Best model saved min loss 0.38209158182144165
In epoch 10, loss: 0.41835013031959534
Epoch 13: Best model saved min loss 0.3803039491176605
Epoch 14: Best model saved min loss 0.3647315800189972
In epoch 15, loss: 0.35424262285232544
Epoch 15: Best model saved min loss 0.35424262285232544
Epoch 16: Be

100%|██████████| 10867/10867 [00:01<00:00, 6503.07it/s]


Train recall@100 0.6104862324571897
Test recall@100 0.5978765462443362


In [26]:
all_nodes_cols = list(nodes.columns.values)

In [ ]:
node

In [28]:
all_nodes_cols = [
 'price_power_transform',
 'avg_rating',
 'no_of_ratings',
 'rank_num_power_transform',
 'label_1',
 'label_2',
 'label_3',
 'label_4',
 'label_5',
 'label_6',
 'label_7',
 'label_8',
 'label_9',
 'label_10',
 'label_11',
 'label_12',
 'label_13',
 'cluster_group_cluster 0',
 'cluster_group_cluster 1',
 'cluster_group_cluster 2',
 'cluster_group_cluster 3',
 'cluster_group_cluster 4',
 'cluster_group_cluster 5',
 'cluster_group_cluster 6',
 'cluster_group_cluster 7',
 'cluster_group_cluster 8',
 'cluster_group_cluster 9',
 'clothing_type_Active',
 'clothing_type_Jackets & Coats',
 'clothing_type_Jeans',
 'clothing_type_Pants',
 'clothing_type_Shirts',
 'clothing_type_Shorts',
 'clothing_type_Sleep & Lounge',
 'clothing_type_Socks',
 'clothing_type_Swim',
 'clothing_type_Underwear']

only_text = ['cluster_group_cluster 0',
 'cluster_group_cluster 1',
 'cluster_group_cluster 2',
 'cluster_group_cluster 3',
 'cluster_group_cluster 4',
 'cluster_group_cluster 5',
 'cluster_group_cluster 6',
 'cluster_group_cluster 7',
 'cluster_group_cluster 8',
 'cluster_group_cluster 9',]

only_product_data = [
 'price_power_transform',
 'avg_rating',
 'no_of_ratings',
 'rank_num_power_transform',
 'label_1',
 'label_2',
 'label_3',
 'label_4',
 'label_5',
 'label_6',
 'label_7',
 'label_8',
 'label_9',
 'label_10',
 'label_11',
 'label_12',
 'label_13',
 'clothing_type_Active',
 'clothing_type_Jackets & Coats',
 'clothing_type_Jeans',
 'clothing_type_Pants',
 'clothing_type_Shirts',
 'clothing_type_Shorts',
 'clothing_type_Sleep & Lounge',
 'clothing_type_Socks',
 'clothing_type_Swim',
 'clothing_type_Underwear']

In [ ]:
# nodes_new = nodes.copy()
# nodes_new['temp'] = 0

In [12]:
IMAGE_MODEL = 'inception'
model,g,nodes,edges,train_pos_u, train_pos_v,test_pos_u, test_pos_v,train_neg_u, train_neg_v,test_neg_u, test_neg_v,train_g,test_g = train_model(mlflow=mlflow,image_model=IMAGE_MODEL,model_params = {'aggregator_type': 'lstm', 'dropout': 0.42051995958759714, 'h_feats': 16, 'lr': 0.022254491166903718},
                                                                                                                                          run_name=f'{IMAGE_MODEL}_no_data_actual',
                                                                                                                                          n_epochs=100,
                                                                                                                                          image_included=False,
                                                                                                                                          split_ratio=0.3,umap_dt={'n_components':100,'n_neighbors':50,'min_dist':0.3},patience=100,only_image=False,skip_ndata=False,use_bidirectional=True,cols_needed=['no_data'])

Node data shape : (10867, 40)
Edges data shape : (20554, 2)
Size of Input features : 1
node features shape : torch.Size([10867, 1])
##########################
Converting to biderctional Graph
##########################
27998 11999
In epoch 0, loss: 1.8259583711624146
Epoch 0: Best model saved min loss 1.8259583711624146
Epoch 1: Best model saved min loss 1.126548171043396
Epoch 2: Best model saved min loss 0.7936579585075378
Epoch 3: Best model saved min loss 0.6106050610542297
Epoch 4: Best model saved min loss 0.5968731045722961
In epoch 5, loss: 0.5507875084877014
Epoch 5: Best model saved min loss 0.5507875084877014
Epoch 6: Best model saved min loss 0.49040740728378296
Epoch 7: Best model saved min loss 0.47377192974090576
Epoch 8: Best model saved min loss 0.4505481421947479
Epoch 9: Best model saved min loss 0.4395662546157837
In epoch 10, loss: 0.4191192090511322
Epoch 10: Best model saved min loss 0.4191192090511322
Epoch 11: Best model saved min loss 0.41060760617256165
Epoch

100%|██████████| 10867/10867 [00:01<00:00, 7756.56it/s]


Train recall@100 0.6104862324571897
Test recall@100 0.5980210086089479


In [ ]:
IMAGE_MODEL = 'inception'
model,g,nodes,edges,train_pos_u, train_pos_v,test_pos_u, test_pos_v,train_neg_u, train_neg_v,test_neg_u, test_neg_v,train_g,test_g = train_model(mlflow=mlflow,image_model=IMAGE_MODEL,model_params = {'aggregator_type': 'lstm', 'dropout': 0.42051995958759714, 'h_feats': 16, 'lr': 0.022254491166903718},
                                                                                                                                          run_name=f'{IMAGE_MODEL}_no_data_actual',
                                                                                                                                          n_epochs=100,
                                                                                                                                          image_included=False,
                                                                                                                                          split_ratio=0.3,umap_dt={'n_components':100,'n_neighbors':50,'min_dist':0.3},patience=100,only_image=False,skip_ndata=True,use_bidirectional=True,cols_needed=[])

skipping adding node features
Node data shape : (10867, 39)
Edges data shape : (20554, 2)
Size of Input features : 0
node features shape : torch.Size([10867, 39])
##########################
Converting to biderctional Graph
##########################
27998 11999
In epoch 0, loss: 27.480663299560547
Epoch 0: Best model saved min loss 27.480663299560547
Epoch 1: Best model saved min loss 8.636537551879883
Epoch 2: Best model saved min loss 4.44978666305542
Epoch 3: Best model saved min loss 2.210573673248291
Epoch 4: Best model saved min loss 0.9635067582130432
In epoch 5, loss: 0.6672428250312805
Epoch 5: Best model saved min loss 0.6672428250312805
Epoch 6: Best model saved min loss 0.571540892124176
Epoch 7: Best model saved min loss 0.5396424531936646
Epoch 8: Best model saved min loss 0.5073716044425964
Epoch 9: Best model saved min loss 0.46218207478523254
In epoch 10, loss: 0.4298371374607086
Epoch 10: Best model saved min loss 0.4298371374607086
Epoch 11: Best model saved min loss

100%|██████████| 10867/10867 [00:12<00:00, 850.31it/s]


Train recall@100 0.6104862324571897
Test recall@100 0.5978765462443362


In [31]:
IMAGE_MODEL = 'inception'
model,g,nodes,edges,train_pos_u, train_pos_v,test_pos_u, test_pos_v,train_neg_u, train_neg_v,test_neg_u, test_neg_v,train_g,test_g = train_model(mlflow=mlflow,image_model=IMAGE_MODEL,model_params = {'aggregator_type': 'lstm', 'dropout': 0.42051995958759714, 'h_feats': 16, 'lr': 0.022254491166903718},
                                                                                                                                          run_name=f'{IMAGE_MODEL}_only_product+text_metadata',
                                                                                                                                          n_epochs=100,
                                                                                                                                          image_included=False,
                                                                                                                                          split_ratio=0.3,umap_dt={'n_components':100,'n_neighbors':50,'min_dist':0.3},patience=100,only_image=False,skip_ndata=False,use_bidirectional=True,cols_needed=only_product_data+only_text)

Node data shape : (10867, 39)
Edges data shape : (20554, 2)
Size of Input features : 37
node features shape : torch.Size([10867, 37])
##########################
Converting to biderctional Graph
##########################
27998 11999
In epoch 0, loss: 5483.46240234375
Epoch 0: Best model saved min loss 5483.46240234375
Epoch 1: Best model saved min loss 5445.2421875
Epoch 2: Best model saved min loss 3668.992919921875
Epoch 3: Best model saved min loss 2709.804931640625
Epoch 4: Best model saved min loss 2200.12646484375
In epoch 5, loss: 7776.74609375
Epoch 7: Best model saved min loss 1172.1524658203125
Epoch 9: Best model saved min loss 1013.9771118164062
In epoch 10, loss: 1486.98388671875
Epoch 12: Best model saved min loss 822.5794677734375
In epoch 15, loss: 1935.3697509765625
Epoch 16: Best model saved min loss 286.2039794921875
Epoch 17: Best model saved min loss 271.3336486816406
Epoch 18: Best model saved min loss 211.3053436279297
In epoch 20, loss: 336.4604797363281
Epoch 2

100%|██████████| 10867/10867 [00:01<00:00, 6215.94it/s]


Train recall@100 0.6104862324571897
Test recall@100 0.5978765462443362


In [32]:
IMAGE_MODEL = 'inception'
model,g,nodes,edges,train_pos_u, train_pos_v,test_pos_u, test_pos_v,train_neg_u, train_neg_v,test_neg_u, test_neg_v,train_g,test_g = train_model(mlflow=mlflow,image_model=IMAGE_MODEL,model_params = {'aggregator_type': 'lstm', 'dropout': 0.42051995958759714, 'h_feats': 16, 'lr': 0.022254491166903718},
                                                                                                                                          run_name=f'{IMAGE_MODEL}_only_product_metadata',
                                                                                                                                          n_epochs=100,
                                                                                                                                          image_included=False,
                                                                                                                                          split_ratio=0.3,umap_dt={'n_components':100,'n_neighbors':50,'min_dist':0.3},patience=100,only_image=False,skip_ndata=False,use_bidirectional=True,cols_needed=only_product_data)

Node data shape : (10867, 39)
Edges data shape : (20554, 2)
Size of Input features : 27
node features shape : torch.Size([10867, 27])
##########################
Converting to biderctional Graph
##########################
27998 11999
In epoch 0, loss: 5387.38134765625
Epoch 0: Best model saved min loss 5387.38134765625
Epoch 1: Best model saved min loss 3287.942138671875
Epoch 4: Best model saved min loss 2835.164794921875
In epoch 5, loss: 5583.015625
Epoch 7: Best model saved min loss 2052.161865234375
Epoch 8: Best model saved min loss 1329.1650390625
In epoch 10, loss: 944.9027709960938
Epoch 10: Best model saved min loss 944.9027709960938
Epoch 11: Best model saved min loss 588.4091796875
Epoch 12: Best model saved min loss 488.39837646484375
Epoch 13: Best model saved min loss 364.8069763183594
In epoch 15, loss: 1105.9344482421875
Epoch 16: Best model saved min loss 290.2758483886719
Epoch 17: Best model saved min loss 201.22207641601562
Epoch 19: Best model saved min loss 156.33

100%|██████████| 10867/10867 [00:01<00:00, 6962.82it/s]


Train recall@100 0.40168960175532337
Test recall@100 0.41524177109963795


In [33]:
IMAGE_MODEL = 'inception'
model,g,nodes,edges,train_pos_u, train_pos_v,test_pos_u, test_pos_v,train_neg_u, train_neg_v,test_neg_u, test_neg_v,train_g,test_g = train_model(mlflow=mlflow,image_model=IMAGE_MODEL,model_params = {'aggregator_type': 'lstm', 'dropout': 0.42051995958759714, 'h_feats': 16, 'lr': 0.022254491166903718},
                                                                                                                                          run_name=f'{IMAGE_MODEL}_only_text_data',
                                                                                                                                          n_epochs=100,
                                                                                                                                          image_included=False,
                                                                                                                                          split_ratio=0.3,umap_dt={'n_components':100,'n_neighbors':50,'min_dist':0.3},patience=100,only_image=False,skip_ndata=False,use_bidirectional=True,cols_needed=only_text)

Node data shape : (10867, 39)
Edges data shape : (20554, 2)
Size of Input features : 10
node features shape : torch.Size([10867, 10])
##########################
Converting to biderctional Graph
##########################
27998 11999
In epoch 0, loss: 0.7939828634262085
Epoch 0: Best model saved min loss 0.7939828634262085
Epoch 1: Best model saved min loss 0.6304920315742493
Epoch 2: Best model saved min loss 0.5162049531936646
Epoch 3: Best model saved min loss 0.4277537167072296
Epoch 4: Best model saved min loss 0.36588162183761597
In epoch 5, loss: 0.3391372263431549
Epoch 5: Best model saved min loss 0.3391372263431549
Epoch 6: Best model saved min loss 0.33505338430404663
Epoch 7: Best model saved min loss 0.32757723331451416
Epoch 8: Best model saved min loss 0.31571924686431885
Epoch 9: Best model saved min loss 0.3055313527584076
In epoch 10, loss: 0.29426226019859314
Epoch 10: Best model saved min loss 0.29426226019859314
Epoch 11: Best model saved min loss 0.2800644636154175

100%|██████████| 10867/10867 [00:13<00:00, 777.70it/s]


Train recall@100 0.706836377593844
Test recall@100 0.6619065635203956


In [29]:
IMAGE_MODEL = 'inception'
model,g,nodes,edges,train_pos_u, train_pos_v,test_pos_u, test_pos_v,train_neg_u, train_neg_v,test_neg_u, test_neg_v,train_g,test_g = train_model(mlflow=mlflow,image_model=IMAGE_MODEL,model_params = {'aggregator_type': 'lstm', 'dropout': 0.42051995958759714, 'h_feats': 16, 'lr': 0.022254491166903718},
                                                                                                                                          run_name=f'{IMAGE_MODEL}_all_data',
                                                                                                                                          n_epochs=100,
                                                                                                                                          image_included=True,
                                                                                                                                          split_ratio=0.3,umap_dt={'n_components':100,'n_neighbors':50,'min_dist':0.3},patience=100,only_image=False,skip_ndata=False,use_bidirectional=True,cols_needed=all_nodes_cols)

c:\Users\Kartik\anaconda3\envs\tf\lib\site-packages\umap\umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


Node data shape : (10867, 139)
Edges data shape : (20554, 2)
Size of Input features : 137
node features shape : torch.Size([10867, 137])
##########################
Converting to biderctional Graph
##########################
27998 11999
In epoch 0, loss: 2142.427490234375
Epoch 0: Best model saved min loss 2142.427490234375
Epoch 1: Best model saved min loss 1161.380126953125
Epoch 3: Best model saved min loss 268.8382263183594
Epoch 4: Best model saved min loss 197.7301025390625
In epoch 5, loss: 53.578372955322266
Epoch 5: Best model saved min loss 53.578372955322266
Epoch 8: Best model saved min loss 18.036212921142578
Epoch 9: Best model saved min loss 6.173149108886719
In epoch 10, loss: 3.4562737941741943
Epoch 10: Best model saved min loss 3.4562737941741943
Epoch 11: Best model saved min loss 2.613926410675049
Epoch 12: Best model saved min loss 2.0599491596221924
In epoch 15, loss: 1.569186806678772
Epoch 15: Best model saved min loss 1.569186806678772
Epoch 16: Best model save

100%|██████████| 10867/10867 [00:01<00:00, 6643.57it/s]


Train recall@100 0.6104862324571897
Test recall@100 0.5978765462443362
